In [ ]:
from google.colab import auth
!gcloud auth activate-service-account your_account --key-file="path_to_the_key"
# add your key here 

In [ ]:
!gcloud auth list
!gcloud config set account your_account
# activate your google cloud account here

In [ ]:
1
from google.colab import auth
# auth.authenticate_user()
!gcloud init
!gcloud auth list

In [ ]:
!gcloud auth list

               Credentialed Accounts
ACTIVE  ACCOUNT
*       lgx-602@gpt3-363020.iam.gserviceaccount.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [ ]:
#@title Setup
%tensorflow_version 2.x
!git clone https://github.com/EleutherAI/GPTNeo
%cd GPTNeo
!pip3 install -q -r requirements.txt
pretrained_model = None
dataset = None

In [ ]:
#@title Setup
%tensorflow_version 2.x
!git clone https://github.com/EleutherAI/GPTNeo
%cd GPTNeo
!pip3 install google-api-python-client
!pip3 install jsonlines
!pip3 install lm_dataformat
!pip3 install mesh-tensorflow==0.1.18
!pip3 install numpy==1.21.0
!pip3 install oauth2client
!pip3 install ortools
!pip3 install pytest
!pip3 install sacred
!pip3 install tensorflow==2.5.1
!pip3 install tensorflow-datasets==3.2.1
!pip3 install tokenizers==0.9.4
!pip3 install transformers==4.1.1
!pip3 install tpunicorn
# !pip3 install -q -r requirements.txt
!pip3 install absl-py
!pip3 install ftfy
!pip3 install sacred
!pip3 install pymongo
# !pip3 install xarray-einstats==0.2.2
# !pip3 install jax==0.3.17
# !pip3 install cmdstanpy==1.0.7
pretrained_model = None
dataset = None

# New Section

In [ ]:
path_to_cloud_bucket = 'gs://gpt3_demo/' #@param {type:"string"}

Set Up Dataset 1 

In [ ]:
import os
dataset = "Custom3" #@param ["Sampling_Only", "OpenWebText", "YoutubeSubtitles", "HackerNews", "NIHExporter", "Custom3", "Custom2"]

if dataset == "Sampling_Only":
  pass
elif dataset == "Custom3":
  os.makedirs('data', exist_ok=True)
  dataset_path = 'data'
  dataset_name = 'code_to_requirement'
  out_name = dataset_name + "_tokenized"
elif dataset == 'YoutubeSubtitles':
  os.makedirs('data', exist_ok=True)
  # !wget https://the-eye.eu/public/AI/pile_preliminary_components/yt_subs.jsonl.zst -O data/yt_subs.jsonl.zst
  dataset_path = 'data'
  dataset_name = 'ytsubs'
  out_name = dataset_name + "_tokenized"

Set Up Dataset 2

In [ ]:
# Select a Dataset:
import os
dataset = 'Custom' #@param ["Sampling_Only", "OpenWebText", "YoutubeSubtitles", "HackerNews", "NIHExporter", "Custom"]

if dataset == "Sampling_Only":
  pass
elif dataset == 'OpenWebText':
  !wget https://the-eye.eu/public/AI/pile_preliminary_components/openwebtext2.jsonl.zst.tar -O openwebtext.tar.xz
  !tar xf openwebtext.tar.xz
  dataset_path = "openwebtext"
  dataset_name = dataset_path
  out_name = dataset_name + "_tokenized"
elif dataset == 'YoutubeSubtitles':
  os.makedirs('data', exist_ok=True)
  !wget https://the-eye.eu/public/AI/pile_preliminary_components/yt_subs.jsonl.zst -O data/yt_subs.jsonl.zst
  dataset_path = 'data'
  dataset_name = 'ytsubs'
  out_name = dataset_name + "_tokenized"
elif dataset == 'HackerNews':
  os.makedirs('data', exist_ok=True)
  !wget https://the-eye.eu/public/AI/pile_preliminary_components/hn.tar.gz -O data/hn.tar.gz
  dataset_path = 'data'
  dataset_name = 'hackernews'
  out_name = dataset_name + "_tokenized"
elif dataset == "NIHExporter":
  os.makedirs('data', exist_ok=True)
  !wget https://the-eye.eu/public/AI/pile_preliminary_components/NIH_ExPORTER_awarded_grant_text.jsonl.zst -O data/NIH_ExPORTER_awarded_grant_text.jsonl.zst
  dataset_path = 'data'
  os.system('mv NIH_ExPORTER_awarded_grant_text.jsonl.zst ./data')
  dataset_name = 'nihexporter'
  out_name = dataset_name + "_tokenized"
elif dataset == "Custom":
  os.makedirs('data', exist_ok=True)
  dataset_path = 'data2'
  dataset_name = 'code_to_uc'
  out_name = dataset_name + "_tokenized"
else:
  raise NotImplementedError('please select from available options: ["OpenWebText", "YoutubeSubtitles", "HackerNews", "NIHExporter", "Custom"]')


**Tokenize and Upload Data
(skip this step if you are sampling from a pre-trained model).**

In [ ]:
# Tokenize Data
!python data/create_tfrecords.py --input_dir /content/GPTNeo/data2/ --name $dataset_name --files_per 100 --output_dir ./tfrecords --write_dataset_config --processes 1

# copy the data to your bucket
if not path_to_cloud_bucket.endswith('/'):
  path_to_cloud_bucket += '/'

copy_loc = path_to_cloud_bucket + "datasets/" + dataset
!gsutil -m cp -r ./tfrecords $copy_loc
!gsutil ls $path_to_cloud_bucket

In [ ]:
%%writefile configs/dataset_configs/Custom.json

{
  "path": "gs://gpt3_demo/datasets/Custom/code_to_uc*.tfrecords",
  "eval_path": "",
  "n_vocab": 50256,
  "tokenizer_is_pretrained": true,
  "tokenizer_path": "gpt2",
  "eos_id": 50256,
  "padding_id": 50257
}

Writing configs/dataset_configs/Custom.json


In [ ]:
%%writefile configs/dataset_configs/YoutubeSubtitles.json

{
  "path": "gs://gpt3_demo/datasets/YoutubeSubtitles/ytsubs*.tfrecords",
  "eval_path": "",
  "n_vocab": 50256,
  "tokenizer_is_pretrained": true,
  "tokenizer_path": "gpt2",
  "eos_id": 50256,
  "padding_id": 50257
}

Writing configs/dataset_configs/YoutubeSubtitles.json


Set Model Configs, do not skip

In [ ]:
%%writefile configs/GPT3_XL3.json

{
    "n_head": 16,
    "n_vocab": 50257,
    "embed_dropout": 0,
    "lr": 0.0002,
    "lr_decay": "cosine",
    "warmup_steps": 3000,
    "beta1": 0.9,
    "beta2": 0.95,
    "epsilon": 1e-8,
    "opt_name": "adam",
    "weight_decay": 0,
    "train_batch_size": 256,
    "attn_dropout": 0,
    "train_steps": 600000,
    "eval_steps": 0,
    "predict_steps": 1,
    "res_dropout": 0,
    "eval_batch_size": 4,
    "predict_batch_size": 1,
    "iterations": 100,
    "n_embd": 2048,
    "datasets": ["Custom"],
    "model": "GPT",
    "model_path": "gs://gpt3_demo/GPT3_XL3",
    "n_ctx": 2048,
    "n_layer": 24,
    "scale_by_depth": true,
    "scale_by_in": false,
    "attention_types" :  [[["global", "local"],12]],
    "mesh_shape": "x:4,y:2",
    "layout": "intermediate_expanded:x,heads:x,vocab:n_vocab,memory_length:y,embd:y",
    "activation_function": "gelu",
    "recompute_grad": true,
    "gradient_clipping": 1.0,
    "tokens_per_mb_per_replica": 2048,
    "precision": "bfloat16"
}

Writing configs/GPT3_XL3.json


In [ ]:

%%writefile configs/GPT3_2-7B.json

{
     "n_head": 32,
    "n_vocab": 50257,
    "embed_dropout": 0,
    "lr": 0.00016,
    "lr_decay": "cosine",
    "warmup_steps": 3000,
    "beta1": 0.9,
    "beta2": 0.95,
    "epsilon": 1e-8,
    "ada_epsilon1": 1e-30,
    "ada_epsilon2": 1e-3,
    "opt_name": "adam",
    "weight_decay": 0.10,
    "train_batch_size": 512,
    "attn_dropout": 0,
    "train_steps": 286150,
    "eval_steps": 0,
    "predict_steps": 1,
    "res_dropout": 0,
    "eval_batch_size": 128,
    "predict_batch_size": 1,
    "iterations": 500,
    "n_embd": 2560,
    "datasets": ["Custom"],
    "model_path": "gs://gpt3_demo/GPT3_2-7B",
    "n_ctx": 2048,
    "n_layer": 32,
    "scale_by_depth": true,
    "scale_by_in": false,
    "attention_types" :  [[["global"],32]],
    "mesh_shape": "x:128,y:2",
    "layout": "embd:y,batch:x",
    "activation_function": "gelu",
    "recompute_grad": true,
    "gradient_clipping": 1.0,
    "tokens_per_mb_per_replica": 2048

}

Overwriting configs/GPT3_2-7B.json


Training from Scratch

In [ ]:
!python3 main.py --model GPT3_XL3 --steps_per_checkpoint 200 --tpu colab

In [ ]:
# @title Download pretrained model weights:
pretrained_model = 'GPT3_XL' #@param ["GPT3_XL", "GPT3_2-7B"]
!wget -m -np -c -U "eye02" -w 2 -R "index.html*" "https://the-eye.eu/public/AI/gptneo-release/$pretrained_model/"
path_to_local_weights = f"/content/GPTNeo/the-eye.eu/public/AI/gptneo-release/GPT3_XL"

# URL = f"http://eaidata.bmk.sh/data/gptneo-release/{pretrained_model}/"
# FOLDER_NAME = "GPT3_XL"
# !curl $URL | grep -i "</a>" | sed -n 's/.*href="\([^"]*\).*/\1/p' | sed "s|^|$URL|" | xargs -n 1 -P 4 wget -P $pretrained_model
# path_to_local_weights = pretrained_model

In [ ]:
# upload to your bucket
path_to_local_weights = f"/content/GPTNeo/GPTNeo/the-eye.eu/public/AI/gptneo-release/GPT3_XL"
bucket_base = "gs://" + path_to_cloud_bucket.replace('gs://', '').split('/')[0]
!gsutil -m cp -r $path_to_local_weights $bucket_base

In [ ]:
!gsutil ls $bucket_base

In [ ]:
# @title Modify config for colab.

import json
from pprint import pprint
bucket_base = "gs://gpt3_demo/"
path_to_model = "" #@param {type:"string"}
batch_size = 8 #@param {type:"integer"}
dset = ""  #@param {type:"string"}
mesh_shape = "x:4,y:2" #@param {type:"string"}
train_steps = 1000 #@param {type:"integer"}
steps_per_checkpoint = 500 #@param {type:"integer"}
start_step = 400000 if pretrained_model == "GPT3_2-7B" else 362000

if path_to_model == "":
  path_to_model = f'{bucket_base.strip("/")}/{pretrained_model}'
print(f'MODEL PATH: {path_to_model}\n')

if dset == "" and dataset != "Sampling_Only":
  dset = dataset
elif dataset is None and dset == "":
  dset = "pile"

def pad_to_multiple_of(n, mult):
  """
  pads n to a multiple of mult
  """
  extra = n % mult
  if extra > 0:
      n = n + mult - extra
  return n

with open(f'{path_to_local_weights}/config.json', 'r') as f:
  data = json.load(f)
  pprint(data)
  dset_val = [[dset, None, None, None]] if dset != "" else data["datasets"]
  mods = {
          "mesh_shape": mesh_shape,
          "layout": "intermediate_expanded:x,heads:x,memory_length:y,embd:y",
          "model_path": path_to_model,
          "datasets": dset_val,
          "train_steps": start_step + train_steps,
          "eval_steps": 0,
          "train_batch_size": batch_size,
          "predict_batch_size": batch_size
        }
  data.update(mods)
  print('\n--->\n')
  pprint(data)
  with open(f'configs/{pretrained_model}.json', 'w') as outfile:
    json.dump(data, outfile, indent=2)

Training from Scratch

In [ ]:
!python3 main.py --model $pretrained_model --steps_per_checkpoint $steps_per_checkpoint --tpu colab

In [ ]:
%%writefile example_prompt.txt
Transform code to requirement:
code:
package interfacce;
import java.awt.*;
import java.awt.event.*;
import common.utility.*;
import common.def.*;
import java.util.Date;
import moduli.*;

public class VediPrenotazione extends MascheraPrenotazione
{  
 Prenotazione p;    
    RicercaPrenotazione parent;
    char flag;
    
    public VediPrenotazione(Prenotazione pren, Stanza room, RicercaPrenotazione papa)
    {
        super(room,"","","Visualizzazione dei dati della prenotazione","",4, pren.getPensionamento());
        p = pren;
        parent = papa;
        init();
    }
        
    public void init()
    {
     Frame msg;
     
     writeDatiPren(p);
     for(int i=0; i<11; i++)
      testo[i].setEditable(false);
     myCheckbox.setEnabled(false);
     myCheckbox1.setEnabled(false);
     for (int i=0; i<checkboxes.length; i++)
      checkboxes[i].setEnabled(false);
     testo[9].setText(DateUtils.parseDate(DateUtils.giveStringOfDate(p.getDataPren())));     
      panel2.remove(Azione);
    } 
}
requirement:

Overwriting example_prompt.txt


In [ ]:
!python3 main.py --model GPT3_XL3 --steps_per_checkpoint 500 --tpu colab --predict --prompt example_prompt.txt

In [ ]:
!python3 main.py --model GPT3_2-7B --steps_per_checkpoint 500 --tpu colab --predict --prompt example_prompt.txt

Begin Fine-Tuning

In [ ]:
!python3 main.py --model $pretrained_model --steps_per_checkpoint $steps_per_checkpoint --tpu colab

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()